![ChIP-Seq Overview](https://biosakshat.weebly.com/uploads/1/2/5/1/12512943/chip-seq-overview-dna-bound-protein-fragment-dna-immunoprecipitate_orig.jpg)

In [ ]:
#Install Miniconda Python 3 Distribution on my Linux
sh Miniconda3-latest-Linux-x86_64.sh

In [ ]:
#Install snakemake
conda install -c bioconda -c conda-forge snakemake


In [ ]:
conda

In [ ]:
First rule: Mapping

In [ ]:
vi snakefile01

In [ ]:
index = '/home/dell/RefGenome/hg19ebwt/hg19'

rule bowtie_map:
    input:
        fq1 = 'FoxA1.fq',
    output:
        o1 = 'mapped_reads/FoxA1.out',
        o2 = 'mapped_reads/FoxA1.sam',
    shell:
        'bowtie {index} -q {input.fq1} -m 1 -5 10 -3 10 -p 32 -S 2> {output.o1} >{output.o2}'

In [ ]:
snakemake -ns snakefile01

In [ ]:
Results


In [ ]:
snakemake -ps snakefile01

In [ ]:
Results









In [ ]:
%%writefile snakefile01

# AUTHOR: DouTY

'''
This pipeline for ChIP-Seq data analysis, which goes from fastq to bam and bam.bai, ready for igv visualization.
for single-end ChIP-Seq data, and requires a reference index.
'''

samples = 'FoxA1 Input 298-301'

rule all:
    input:
        expand('/home/dell/DouTY/FoxA1/{sample}.fq', sample=samples.split(' '))

rule bowtie_map:
    input:
        index = '/home/dell/RefGenome/hg19ebwt/hg19',
        fq = '/home/dell/DouTY/FoxA1/{sample}.fq'
    output:
        o1 = '/home/dell/DouTY/FoxA1/mapped_reads/{sample}.out',
        o2 = '/home/dell/DouTY/FoxA1/mapped_reads/{sample}.sam',
    shell:
        'bowtie -m 1 -5 10 -3 10 -p 32 -s {input.index} -q {input.fq} 2> {output.o1} >{output.o2}'
        


In [ ]:
!snakemake -ns snakefile01


In [ ]:
!snakemake -ps snakefile01


rule samtools_sort:
    input:
        '/home/dell/DouTY/FoxA1/mapped_reads/{sample}.sam'
    output:
        '/home/dell/DouTY/FoxA1/sorted_reads/{sample}.bam'
    shell:
        'samtools sort -@ 8 -o {input} {output}'
 

   


In [ ]:
rule samtools_index:
    input:
        '/home/dell/DouTY/FoxA1/sorted_reads/{sample}.bam' 
    output:
        '/home/dell/DouTY/FoxA1/sorted_reads/{sample}.bam.bai'
    shell:
        'samtools index {input}' 


In [ ]:
#Data Visualization using Deeptools

plotEnrichment [-h] --bamfiles FoxA1.bam 298-301.bam Input.bam

![Enrichment](Enrichment.png)

In [ ]:
#Data Visualization using the Integrative Genomics Viewer (IGV)
